In [ ]:
import os 
import anthropic
import google.generativeai
from openai import OpenAI
from IPython.display import display, Markdown, update_display
from dotenv import load_dotenv
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()
gemini = google.generativeai.configure()

In [ ]:
system_prompt = "You are a helpful assistant"

def get_response_from_llm(user_prompt, model):
    MESSAGES = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]

    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',                  
        messages = MESSAGES,
        temperature = 0.4
    )
    return response.choices[0].message.content

In [ ]:
view = gr.Interface(fn = get_response_from_llm, inputs = 'textbox', outputs = 'textbox')
view.launch()

In [ ]:
view = gr.Interface(fn = get_response_from_llm, inputs = 'textbox', outputs = 'textbox', 
    allow_flagging= 'never')
view.launch(share=True)

In [ ]:
def get_stream_response_from_gpt(user_prompt):
    MESSAGES = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}
    ]
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = MESSAGES,
        temperature = 0.4,
        stream = True
    )
    result = ''
    for chunk in response:
        result += chunk.choices[0].delta.content or ''
        yield result

In [ ]:
def get_stream_response_from_claude(user_prompt):
    MESSAGES = [
        {'role': 'user', 'content': user_prompt}
    ]
    response = claude.messages.stream(
        model = 'gpt-4o-mini',
        messages = MESSAGES,
        temperature = 0.4,
        system= system_prompt
    )
    result = ''
    with response as stream:
        for chunk in stream.text_stream:
            result += chunk or ''
            yield result
           
         

In [ ]:
def response_from_streaming_model(user_prompt, model):
    if model == 'GPT':
        response = get_stream_response_from_gpt(user_prompt)
    elif model == 'Claude':
        response = get_stream_response_from_claude(user_prompt)
    else:
        raise ValueError('Invalid Model')
    yield from response

In [ ]:
view = gr.Interface(
    fn = response_from_streaming_model,
    inputs = [gr.Textbox(label='Your Message:'),
            gr.Dropdown(['GPT', 'Claude'], label='Select Model')],
    outputs = gr.Markdown(label='Response:'),
    allow_flagging = 'never'
)
view.launch()